# Compare and Analyze Data on COVID-19 Infections Provided by the [Robert Koch Institute (RKI)](https://www.rki.de/EN/Home/homepage_node.html)

The [Robert Koch Institute (RKI)](https://www.rki.de/EN/Home/homepage_node.html) is the federal government agency responsible for disease control and prevention in Germany. Is publishes data about COVID-19 for all of Germany and uses various channels for that (see [this](https://www.rki.de/EN/Content/infections/epidemiology/outbreaks/COVID-19/COVID19.html) page for an overview).

This notebook has been created for analyzing and comparing data from two different sources that are updated by the RKI daily, but have a different level of detail. The main objective is to understand how the very fine grained numbers provided via github can be aggregated such that they match what is shown in the 
[RKI's COVID-19 dashboard](https://corona.rki.de/).

## Preliminaries

In [1]:
import json

import datetime as dt
import numpy as np
import pandas as pd

import local_constants as LC
from urllib.request import urlopen

## Load and Analyze Data From the [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/)

For all German districts, up-to-date COVID-19 data is available via [this](https://opendata.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0) page. This data appears to be the basis for the [COVID-19 dashboard](https://corona.rki.de/).

### Read Data

In [2]:
# load main data, but restrict the created dataframe to the most relevant columns 

RKI_ARCGIS_COVID_BY_DISTRICT = \
    pd.read_csv(LC.RKI_ARCGIS_URL, usecols=list(LC.RKI_ARCGIS_COLUMN_NAME_MAPPER.keys()))\
        .rename(columns=LC.RKI_ARCGIS_COLUMN_NAME_MAPPER)\
        .sort_values(by="district ID")

RKI_ARCGIS_COVID_BY_DISTRICT = RKI_ARCGIS_COVID_BY_DISTRICT.convert_dtypes()

In [3]:
most_frequent_date = RKI_ARCGIS_COVID_BY_DISTRICT[["update time", "district ID"]].groupby(by="update time").count().sort_values(by="district ID", ascending=False).index[0]

In [4]:

RKI_ARCGIS_COVID_BY_DISTRICT["update time"].fillna(value=most_frequent_date, axis="index", inplace=True)


In [5]:

RKI_ARCGIS_COVID_BY_DISTRICT["update time"] = pd.to_datetime(RKI_ARCGIS_COVID_BY_DISTRICT["update time"], format="%d.%m.%Y, %H:%M Uhr")


In [6]:
RKI_ARCGIS_COVID_BY_DISTRICT.set_index("district ID", inplace=True)

In [7]:

# add column for the number of deaths within the last seven days adjusted to a population size of 100.000 people

RKI_ARCGIS_COVID_BY_DISTRICT["deaths last 7 days per 100k"] = \
    10**5 * RKI_ARCGIS_COVID_BY_DISTRICT["deaths last 7 days"] \
    / RKI_ARCGIS_COVID_BY_DISTRICT["population"]

In [8]:
# print date of most recent entry in the loaded data

print(RKI_ARCGIS_COVID_BY_DISTRICT["update time"].max().strftime("last update is from %Y-%m-%d"))

last update is from 2021-10-03


## Load and Analyze RKI's Data on COVID-19 From [GitHub](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland)

Repository ["SARS-CoV-2 Infektionen in Deutschland"](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland) (SARS-CoV-2 Infections in Germany) contains
up-to-date numbers of COVID-19 cases in Germany. The data appears to be what is reported by the districts to the RKI as it lists new cases based on the reporting date, beginning of the disease (reference date), age group, sex and district.

In [Readme.md](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/blob/master/Readme.md), an explanation for the data is provided (in German). 

### Read Metadata

The RKI publishes metadata based on [zenodo's](https://about.zenodo.org/) JSON format. Here, it is used to detect the publication date of the data. Typically, this is shortly after 3 AM of the current day (local time in Germany). 

In [9]:
RKI_GITHUB_METADATA = json.loads(urlopen(LC.RKI_GITHUB_RAW_DATA_BASE_URL + LC.RKI_GITHUB_ZENODO_REL_URL).read())
RKI_GITHUB_PUBLICATION_DATE = pd.to_datetime(RKI_GITHUB_METADATA["publication_date"])
print(f"publication date is {RKI_GITHUB_PUBLICATION_DATE:%Y-%m-%d %H:%M:%S%z}")

publication date is 2021-10-03 03:10:36+0200


### Read Data

Load data describing COVID-19 infections and deaths etc.

In [10]:
RKI_GITHUB_COVID_INFECTIONS = pd.read_csv(LC.RKI_GITHUB_RAW_DATA_BASE_URL + LC.RKI_GITHUB_COVID_INFECTIONS_REL_URL, 
                                    converters=LC.RKI_GITHUB_VALUE_CONVERTERS)\
                                .rename(columns=LC.RKI_GITHUB_COLUMN_NAME_MAPPER)\
                                .astype(LC.RKI_GITHUB_COLUMN_TYPES_MAPPER)

### Trying to Understand [Readme.md](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/blob/master/Readme.md)

According to the text in file [Readme.md](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/blob/master/Readme.md), are the values for the number of 
infected (column `cases` - `AnzahlFall` in the original data), deceased (column `deaths` - column `AnzahlTodesfall` before renaming) and recovered 
(column `recovered` - `AnzahlGenesen` before renaming) people, **[natural numbers](https://en.wikipedia.org/wiki/Natural_number)** (i.e. elements of {1,2,3 ...}). 

However, negative values seem to occur in the data. 

In [11]:
# show that there are rows, for which the value in one of the columns "cases", "deaths" or "recovered" is negative

((RKI_GITHUB_COVID_INFECTIONS["cases"] < 0) |  \
    (RKI_GITHUB_COVID_INFECTIONS["deaths"] < 0) |  \
        (RKI_GITHUB_COVID_INFECTIONS["recovered"] < 0))\
            .value_counts()[True] > 0

True

Hence, despite what's stated in `Readme.md`, values of the respective columns are **integers** (i.e. elements of {... -3, -2, -1, 0,1,2,3 ...}) and **not** natural numbers. 

If one wants to determine to total number of COVID-19 cases reported in Germany, `Readme.md` seems to imply that only such rows should be considered for which the value 
column `is new case` (`NeuerFall` in the original data) is not 0, because 0 indicates that the respective case was already reported previously. 

Additionally, one would think that rows for which the values in column `is new case` is -1 should be subtracted, because this value indicates that cases have been falsely reported in the past.

However, it appears that 
$$

 n = -1  \Leftrightarrow c < 0,
$$ 
where $n$ stands for the value in column `is new case` and $c$ for the value in column `cases` in a row. This means that values for such rows should **not be subtracted but added** (since they are negative).

In [12]:
# verify that in all rows of dataframe RKI_GITHUB_COVID_INFECTIONS, the value in column "is new case" is -1 
# iff the value in column "cases" is less than 0

equiv = lambda a,b: ((~a) | b) & ((~b) | a) 
equiv(RKI_GITHUB_COVID_INFECTIONS["is new case"] == -1, RKI_GITHUB_COVID_INFECTIONS["cases"] < 0).all()

True

Hence, it should be correct to simply add all values of column `cases`, where `is new case` is not 0 to get the total number of cases in Germany ...

In [13]:
# print the sum for column "cases", filtered by "is new case" == 1 (i.e. is new case) 

print(f'{RKI_GITHUB_COVID_INFECTIONS.loc[RKI_GITHUB_COVID_INFECTIONS["is new case"] == 1, "cases"].sum():,d}')
RKI_GITHUB_COVID_INFECTIONS.loc[RKI_GITHUB_COVID_INFECTIONS["is new case"] == 1].sort_values(by="reporting date")

6,211


,district ID,age group,sex,reporting date,reference date,is start of desease,is new case,is new death,is new recovered,cases,deaths,recovered
2129664,13073,A60-A79,F,2020-11-03,2020-11-03,0,1,-9,1,1,0,1
1132115,8115,A00-A04,F,2020-12-08,2021-09-29,1,1,-9,-9,1,0,0
472898,5315,A15-A34,F,2021-05-01,2021-05-01,0,1,-9,1,1,0,1
1479699,9162,A15-A34,F,2021-06-28,2021-06-28,0,1,-9,1,1,0,1
1479854,9162,A05-A14,F,2021-07-13,2021-07-13,0,1,-9,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1370289,8335,A15-A34,M,2021-10-02,2021-10-02,0,1,-9,-9,2,0,0
1374413,8335,A15-A34,F,2021-10-02,2021-09-24,1,1,-9,-9,1,0,0
1374444,8335,A15-A34,F,2021-10-02,2021-09-28,1,1,-9,-9,1,0,0
489731,5334,A15-A34,M,2021-10-02,2021-10-02,0,1,-9,-9,1,0,0


... but this number is **way too small**. 

The number that is shown on the dashboard can be retrieved from the data by adding _all positive values_ in column `cases`. 

In [14]:
print("total cases for Germany:")
# print the sum for all positive values in column "cases" 
print(f' GitHub: {RKI_GITHUB_COVID_INFECTIONS.loc[RKI_GITHUB_COVID_INFECTIONS["cases"] > 0, "cases"].sum():,d}')
# sum of all cases in the data from ARCGIS
print(f' ARCGIS: {RKI_ARCGIS_COVID_BY_DISTRICT["cases"].sum():,d}')

total cases for Germany:
 GitHub: 4,252,300
 ARCGIS: 4,252,300


### Compute Totals

Based on te interpretation of the data described above, the sum of columns `cases`, `deaths` and `recovered` is calculated for each district.

For the sake of convenience, a new dataframe is defined, in which negative values for the number of COVID-19 cases, deaths and recovered patients are set to zero. This is used later for the aggregation of data.

In [15]:
# define dataframe that without the negative values in RKI_GITHUB_COVID_INFECTIONS

RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES = \
    pd.concat([RKI_GITHUB_COVID_INFECTIONS[["district ID", "age group", "sex", "reporting date", "reference date"]], 
              RKI_GITHUB_COVID_INFECTIONS[["cases", "deaths", "recovered"]].apply(lambda a: np.maximum(a,0))], 
              axis="columns")

In [16]:
#  sum up "cases", "deaths", "recovered" for each district
RKI_GITHUB_COVID_BY_DISTRICT_TOTALS = \
    RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES[["district ID", "cases", "deaths", "recovered"]].groupby(by="district ID").sum()

# copy population size for each district from RKI_ARCGIS_COVID_BY_DISTRICT
RKI_GITHUB_COVID_BY_DISTRICT_TOTALS["population"] = RKI_ARCGIS_COVID_BY_DISTRICT["population"]

### Compute Numbers per 100K People 

The sums in columns `cases`, `deaths` and `recovered` are normalized by the population size of the district. 
For the district's population size, the data from the [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/) is used.

In [17]:
# define a new dataframe by dividing the totals by the population size of each district and multiplying that with 100,000

RKI_GITHUB_COVID_BY_DISTRICT_PER_100K = \
    pd.DataFrame(data=10**5 * RKI_GITHUB_COVID_BY_DISTRICT_TOTALS[["cases", "deaths", "recovered"]].values \
                 / np.array(3 * [RKI_GITHUB_COVID_BY_DISTRICT_TOTALS["population"].values]).T,
                 index=RKI_GITHUB_COVID_BY_DISTRICT_TOTALS.index,
                 columns=["cases per 100k", "deaths per 100k", "recovered per 100k"])

### Compute Totals for the Last Seven Days

The values in columns `cases`, `deaths` and `recovered` of the last seven days are summed up.

In [18]:
# define the date of the earlist data that should be considered
number_of_days = 7 
cut_off_date = np.datetime64(RKI_GITHUB_PUBLICATION_DATE.date() - dt.timedelta(days=number_of_days))

# compute sums for data that has been reported on or after the cut_off_date
RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS = \
        RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES\
                .loc[RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES["reporting date"] >= cut_off_date]\
                .groupby(by="district ID").sum()\
                .rename(columns={"cases": "cases last 7 days", "deaths": "deaths last 7 days", "recovered": "recovered last 7 days"})

# ensure that there is data for each district by filling missing data with zeros
# define a dataframe containing the most recent data of each district
df = RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES[["district ID", "reporting date"]].sort_values(by=["district ID", "reporting date"])\
        .groupby(["district ID"]).last()

missing_rows = df.loc[df["reporting date"] < cut_off_date].index.shape[0]
if  missing_rows > 0:
    # there are districts that have not reported data within the last 7 days
    # create dataframe containing the zeroes filling the missing data 
    ddf = pd.DataFrame(data=np.zeros((missing_rows, RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.shape[1]), dtype=np.int64), 
                       index=df.loc[df["reporting date"] < cut_off_date].index,
                       columns=RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.columns)
    # append zeros
    RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS = RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.append(ddf)

### Compute Numbers for the Last Seven Days per 100K People 

Normalize the sums for the last seven days by the districts' population size 

In [19]:
# define a new dataframe computing values by dividing the totals for the last seven days by the population size 
# of each district and multiplying it with 100,000
 
RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_PER_100K = \
    pd.DataFrame(data=10**5 * RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.values / np.array(3 * \
        [RKI_GITHUB_COVID_BY_DISTRICT_TOTALS.loc[RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.index, "population"]]).T,
        index=RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.index,
        columns=["cases last 7 days per 100k", "deaths last 7 days per 100k", "recovered last 7 days per 100k"])

### Create a DataFrame Containing all Values Derived From the COVID-19 Data from GitHub

In [20]:
# combine all of the dataframes with data that has been derived from RKI_GITHUB_COVID_INFECTIONS into one dataframe, 
# only columns "district name" and "state name" are taken from RKI_ARCGIS_COVID_BY_DISTRICT

RKI_GITHUB_COVID_BY_DISTRICT = \
    pd.concat([RKI_ARCGIS_COVID_BY_DISTRICT[["district name", "state name"]],
               RKI_GITHUB_COVID_BY_DISTRICT_TOTALS, 
               RKI_GITHUB_COVID_BY_DISTRICT_PER_100K, 
               RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS,
               RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_PER_100K], axis="columns")

## Verify that Derived Data From the [COVID-19 Data from GitHub](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland) and the Data From [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/) are (More or Less) Identical

In [21]:
EPSILON = 10**-11 # threshold for treating float values as zero
common_columns = list(set(RKI_ARCGIS_COVID_BY_DISTRICT.columns) & set(RKI_GITHUB_COVID_BY_DISTRICT.columns)) 
common_numerical_columns = [c for c in common_columns if (RKI_GITHUB_COVID_BY_DISTRICT[c].dtypes == np.dtype('float64') \
                                                            or RKI_GITHUB_COVID_BY_DISTRICT[c].dtypes == np.dtype('int64'))]

# return True, if all the differences between the absolute values in the common numerical columns is smaller than the threshold

np.amax(np.abs(RKI_ARCGIS_COVID_BY_DISTRICT[common_numerical_columns].values \
    - RKI_GITHUB_COVID_BY_DISTRICT[common_numerical_columns].values)) < EPSILON

/home/stefan/RKI-COVID-15-Data-Sources-Analysis/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


True

## Show Some Data

Based on the result of the verification above, it seems that the interpretation of the data that is provided via GitHub is correct ... or at least identical with what is shown
in the COVID-19 Dashboard. Hence, the following list the most affected districts in Germany can be assumed to be correct.

In [22]:
n = 30
RKI_GITHUB_COVID_BY_DISTRICT.sort_values(by="cases last 7 days per 100k", ascending=False)\
    .head(n).style.hide_index().format(LC.FORMAT_MAPPER)

district name,state name,cases,deaths,recovered,population,cases per 100k,deaths per 100k,recovered per 100k,cases last 7 days,deaths last 7 days,recovered last 7 days,cases last 7 days per 100k,deaths last 7 days per 100k,recovered last 7 days per 100k
SK Bremerhaven,Bremen,"6,190",111,"5,151","113,557","5,451.0",97.748267,"4,536.0",295,1,0,259.8,0.881,0.0
LK Traunstein,Bayern,"13,140",220,"12,063","177,485","7,403.4",123.954137,"6,796.6",438,0,1,246.8,0.000,0.6
LK Berchtesgadener Land,Bayern,"7,449",102,"6,998","106,327","7,005.7",95.930479,"6,581.6",249,0,3,234.2,0.000,2.8
LK Rosenheim,Bayern,"16,812",473,"15,069","261,721","6,423.6",180.726804,"5,757.7",574,0,2,219.3,0.000,0.8
SK Pforzheim,Baden-Württemberg,"9,165",210,"8,288","126,016","7,272.9",166.645505,"6,576.9",266,1,1,211.1,0.794,0.8
SK Schweinfurt,Bayern,"3,712",96,"3,457","53,319","6,961.9",180.048388,"6,483.6",87,0,1,163.2,0.000,1.9
LK Schwarzwald-Baar-Kreis,Baden-Württemberg,"11,784",218,"11,064","212,872","5,535.7",102.408959,"5,197.5",347,0,2,163.0,0.000,0.9
SK Heilbronn,Baden-Württemberg,"10,165",141,"9,674","126,458","8,038.2",111.499470,"7,650.0",199,1,3,157.4,0.791,2.4
SK Offenbach,Hessen,"11,089",185,"10,277","130,892","8,471.9",141.337897,"7,851.5",203,0,2,155.1,0.000,1.5
SK Nürnberg,Bayern,"38,929",979,"36,406","515,543","7,551.1",189.896866,"7,061.7",792,0,0,153.6,0.000,0.0


Show sums by state

In [23]:
base_columns = ["cases", "deaths", "recovered"]
last_7_days_columns = [c + " last 7 days" for c in base_columns]
index_column = "state name"
columns = [index_column, "population"] + base_columns + last_7_days_columns
RKI_GITHUB_COVID_BY_STATE = RKI_GITHUB_COVID_BY_DISTRICT[columns].groupby(by=index_column).sum()

per_100k_columns = [c + " per 100k" for c in base_columns]
last_7_days_per_100k_columns = [c + " per 100k" for c in last_7_days_columns]
RKI_GITHUB_COVID_BY_STATE[per_100k_columns + last_7_days_per_100k_columns] = \
    10**5 * RKI_GITHUB_COVID_BY_STATE[base_columns + last_7_days_columns].values\
        / np.array(6 * [RKI_GITHUB_COVID_BY_STATE["population"]]).T
RKI_GITHUB_COVID_BY_STATE.style.format(LC.FORMAT_MAPPER)

,population,cases,deaths,recovered,cases last 7 days,deaths last 7 days,recovered last 7 days,cases per 100k,deaths per 100k,recovered per 100k,cases last 7 days per 100k,deaths last 7 days per 100k,recovered last 7 days per 100k
state name,,,,,,,,,,,,,
Baden-Württemberg,"11,103,043","576,245","10,691","544,789","9,189",11,116,"5,190.0",96.288918,"4,906.7",82.8,0.099,1.0
Bayern,"13,140,183","734,011","15,674","692,891","11,712",6,86,"5,586.0",119.282966,"5,273.1",89.1,0.046,0.7
Berlin,"3,657,463","207,130","3,628","196,414","2,758",0,61,"5,663.2",99.194442,"5,370.2",75.4,0.000,1.7
Brandenburg,"2,531,071","116,587","3,852","110,221",972,0,13,"4,606.2",152.188540,"4,354.7",38.4,0.000,0.5
Bremen,"680,130","33,153",510,"30,799",783,1,4,"4,874.5",74.985665,"4,528.4",115.1,0.147,0.6
Hamburg,"1,852,478","92,032","1,741","86,260","1,169",3,7,"4,968.0",93.982223,"4,656.5",63.1,0.162,0.4
Hessen,"6,293,154","334,707","7,753","314,393","3,997",4,43,"5,318.6",123.197367,"4,995.8",63.5,0.064,0.7
Mecklenburg-Vorpommern,"1,610,774","49,195","1,202","46,730",813,0,11,"3,054.1",74.622511,"2,901.1",50.5,0.000,0.7
Niedersachsen,"8,003,421","301,713","5,963","287,322","3,637",1,51,"3,769.8",74.505640,"3,590.0",45.4,0.012,0.6


Equally, the following totals for all of Germany appear correct

In [24]:
columns = ["population", "cases", "cases last 7 days", "deaths", "deaths last 7 days", "recovered", "recovered last 7 days"]
RKI_GITHUB_COVID_BY_DISTRICT[columns].sum().to_frame().T.style.hide_index().format(LC.FORMAT_MAPPER)

population,cases,cases last 7 days,deaths,deaths last 7 days,recovered,recovered last 7 days
"83,106,436","4,252,300","53,366","93,786",36,"4,024,269",659


Likewise, the sums for the last 7 days per 100,000 people can be computed

In [25]:
columns = ["cases last 7 days", "deaths last 7 days", "recovered last 7 days"]
(10**5 * RKI_GITHUB_COVID_BY_DISTRICT[columns].sum() / RKI_GITHUB_COVID_BY_DISTRICT["population"].sum())\
    .to_frame().T.rename(columns={c:c+" per 100k" for c in columns}).style.hide_index().format(LC.FORMAT_MAPPER)

cases last 7 days per 100k,deaths last 7 days per 100k,recovered last 7 days per 100k
64.2,0.043,0.8


For the increase in the numbers of cases, deaths and recovered people within the last day, the data for the previous day is loaded and subtracted from the totals of the current day.

In [26]:
PREVIOUS_DAY = pd.Timestamp(RKI_GITHUB_PUBLICATION_DATE.date() - dt.timedelta(days=1))
RKI_GITHUB_COVID_INFECTIONS_PREVIOUS_DAY \
    = pd.read_csv(LC.RKI_GITHUB_RAW_DATA_BASE_URL + "/Archiv" + \
                  PREVIOUS_DAY.strftime("/%Y-%m-%d_Deutschland_SarsCov2_Infektionen.csv"), 
                  converters=LC.RKI_GITHUB_VALUE_CONVERTERS)\
        .rename(columns=LC.RKI_GITHUB_COLUMN_NAME_MAPPER)\
        .astype(LC.RKI_GITHUB_COLUMN_TYPES_MAPPER)

RKI_GITHUB_COVID_INFECTIONS_PREVIOUS_DAY_WITHOUT_NEGATIVES = \
    pd.concat([RKI_GITHUB_COVID_INFECTIONS_PREVIOUS_DAY[["district ID", "age group", "sex", "reporting date", "reference date"]], 
               RKI_GITHUB_COVID_INFECTIONS_PREVIOUS_DAY[["cases", "deaths", "recovered"]].apply(lambda a: np.maximum(a,0))], 
               axis="columns")
               
(RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES[["cases", "deaths", "recovered"]].sum() 
    - RKI_GITHUB_COVID_INFECTIONS_PREVIOUS_DAY_WITHOUT_NEGATIVES[["cases", "deaths", "recovered"]].sum())\
        .to_frame().T.style.hide_index().format(LC.FORMAT_MAPPER)

cases,deaths,recovered
"6,164",9,"5,787"


### Compute Totals by Age Group and Sex

In order to close this notebook with something that is not mysterious, the total number of cases, deaths and recovered people is computed by age group and sex. 
This is again in sync with the data of the COVID-19 dashboard.

In [27]:
RKI_GITHUB_COVID_BY_AGE_GROUP_AND_SEX_TOTALS = \
    RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES[["age group", "sex", "cases", "deaths", "recovered"]]\
        .groupby(by=["age group", "sex"]).sum()
RKI_GITHUB_COVID_BY_AGE_GROUP_AND_SEX_TOTALS.style.format(LC.FORMAT_MAPPER)